In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import copy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
warnings.filterwarnings(action='ignore')
from epftoolbox.evaluation import MAE, sMAPE

# DNN

In [2]:
df = pd.read_csv(r"Data/df_espana_final.csv")

In [3]:
df

,Date,Price,Exogenous 1,Exogenous 2,Exogenous 3,Exogenous 4
0,2019-01-02 00:00:00,64.03,24914.0,5795.0,0.0,19.500
1,2019-01-02 01:00:00,61.74,22758.0,6460.0,0.0,24.400
2,2019-01-02 02:00:00,53.00,20772.0,6684.0,0.0,15.200
3,2019-01-02 03:00:00,48.23,19690.0,6957.0,0.0,24.300
4,2019-01-02 04:00:00,47.23,19176.0,7292.0,0.0,6.000
...,...,...,...,...,...,...
38659,2023-05-31 19:00:00,78.89,25595.5,2325.8,8196.5,1413.275
38660,2023-05-31 20:00:00,85.00,26170.3,2345.5,5498.0,1340.275
38661,2023-05-31 21:00:00,105.00,26918.3,2253.3,1989.3,1021.550
38662,2023-05-31 22:00:00,119.06,27557.3,2265.3,232.8,681.525


In [4]:
df['Date'] = pd.to_datetime(df.Date)

In [5]:
df = df.fillna(0)

In [6]:
df.isna().sum()

Date           0
Price          0
Exogenous 1    0
Exogenous 2    0
Exogenous 3    0
Exogenous 4    0
dtype: int64

In [7]:
df['Exogenous 2'] = df['Exogenous 2'] + df['Exogenous 3'] + df['Exogenous 4']
df = df.drop(['Exogenous 3', 'Exogenous 4'], axis = 1)

In [8]:
df.index = df.Date

In [9]:
df = df.drop('Date', axis = 1)

In [10]:
df_train = df[df.index < dt.datetime(2022, 1, 1)]
df_test = df[df.index >= dt.datetime(2022, 1, 1)]

In [11]:
df_test.columns

Index(['Price', 'Exogenous 1', 'Exogenous 2'], dtype='object')

In [12]:
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

In [13]:
forecast_dates = forecast.index

In [14]:
from epftoolbox.models import hyperparameter_optimizer

In [15]:
df.to_csv("Data/df_espana_dnn.csv")

In [16]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "df_espana_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 3

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 1

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 25.4 | sMAPE: 34.36 %

Best MAE - Test Dataset
  MAE: 63.3 | sMAPE: 52.87 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 17.5 | sMAPE: 26.65 %

Best MAE - Test Dataset
  MAE: 41.6 | sMAPE: 38.58 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 15.8 | sMAPE: 23.13 %

Best MAE - Test Dataset
  MAE: 39.3 | sMAPE: 35.02 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 15.8 | sMAPE: 23.13 %

Best MAE - Test Dataset
  MAE: 39.3 | sMAPE: 35.02 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 11.1 | sMAPE: 19.05 %

Best MAE - Test Dataset
  MAE: 42.0 | sMAPE: 35.82 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 11.1 | sMAPE: 19.05 %

Best MAE - Test Dataset
  MAE: 42.0 | sMAPE: 35.82 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 11.1 | sMAPE: 19.05 %

Best MAE - Test 



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 6.7 | sMAPE: 11.66 %

Best MAE - Test Dataset
  MAE: 21.9 | sMAPE: 23.09 %


Tested 67/100 iter

In [3]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "df_espana_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_espana_final.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 3
experiment_id = 1

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00



KeyboardInterrupt



In [ ]:
forecast

In [ ]:
forecast.to_csv("Resultados/dnn_espana_final.csv")

# DNN + Estandarización móvil

In [4]:
with open('dict_new_df_espana_final.pkl', 'rb') as handle:
    dict_new_df = pickle.load(handle)

In [5]:
df_nuevo = pd.DataFrame(dict_new_df)

In [6]:
df_nuevo['Date'] = pd.to_datetime(df_nuevo.Date)

In [7]:
df_scalers = pd.DataFrame({'Date':dict_new_df['Date'], 'scaler':dict_new_df['scaler']})

In [8]:
df_scalers

,Date,scaler
0,2019-01-09 00:00:00,StandardScaler()
1,2019-01-09 01:00:00,StandardScaler()
2,2019-01-09 02:00:00,StandardScaler()
3,2019-01-09 03:00:00,StandardScaler()
4,2019-01-09 04:00:00,StandardScaler()
...,...,...
38491,2023-05-31 19:00:00,StandardScaler()
38492,2023-05-31 20:00:00,StandardScaler()
38493,2023-05-31 21:00:00,StandardScaler()
38494,2023-05-31 22:00:00,StandardScaler()


In [9]:
df_nuevo = df_nuevo[['Date', 'Price', 'Exogenous 1', 'Exogenous 2']]

In [10]:
df_nuevo = df_nuevo.set_index('Date')

In [11]:
df_nuevo.to_csv("Data/df_espana_dnn_estandarizacion_movil.csv")

In [19]:
from epftoolbox.models import hyperparameter_optimizer

In [20]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "df_espana_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = None

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 2

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.9 | sMAPE: 118.15 %

Best MAE - Test Dataset
  MAE: 0.9 | sMAPE: 127.10 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.9 | sMAPE: 121.56 %

Best MAE - Test Dataset
  MAE: 0.9 | sMAPE: 129.37 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.6 | sMAPE: 85.10 %

Best MAE - Test Dataset
  MAE: 0.7 | sMAPE: 104.99 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.6 | sMAPE: 85.10 %

Best MAE - Test Dataset
  MAE: 0.7 | sMAPE: 104.99 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.6 | sMAPE: 85.10 %

Best MAE - Test Dataset
  MAE: 0.7 | sMAPE: 104.99 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.6 | sMAPE: 85.10 %

Best MAE - Test Dataset
  MAE: 0.7 | sMAPE: 104.99 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.6 | sMAPE: 85.10 %

Best MAE - Test Datas



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.65 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 84.94 %


Tested 67/100 iterations.

In [12]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "df_espana_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_espana_estandarizacion_movil_final.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = None
experiment_id = 2

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00
2022-01-01 - sMAPE: 63.14%  |  MAE: 0.335
2022-01-02 - sMAPE: 70.99%  |  MAE: 0.309
2022-01-03 - sMAPE: 87.80%  |  MAE: 0.383
2022-01-04 - sMAPE: 100.20%  |  MAE: 0.488
2022-01-05 - sMAPE: 102.23%  |  MAE: 0.562
2022-01-06 - sMAPE: 97.78%  |  MAE: 0.570
2022-01-07 - sMAPE: 92.01%  |  MAE: 0.544
2022-01-08 - sMAPE: 91.32%  |  MAE: 0.538
2022-01-09 - sMAPE: 87.59%  |  MAE: 0.559
2022-01-10 - sMAPE: 86.23%  |  MAE: 0.556
2022-01-11 - sMAPE: 82.48%  |  MAE: 0.539
2022-01-12 - sMAPE: 85.17%  |  MAE: 0.528
2022-01-13 - sMAPE: 86.18%  |  MAE: 0.514
2022-01-14 - sMAPE: 89.29%  |  MAE: 0.512
2022-01-15 - sMAPE: 90.51%  |  MAE: 0.495
2022-01-16 - sMAPE: 90.45%  |  MAE: 0.475
2022-01-17 - sMAPE: 89.54%  |  MAE: 0.480
2022-01-18 - sMAPE: 92.19%  |  MAE: 0.503
2022-01-19 - sMAPE: 93.19%  |  MAE: 0.503
2022-01-20 - sMAPE: 93.42%  |  MAE: 0.513
2022-01-21 - sMAPE: 90.77%  |  MAE: 0.502
2022-01-22 - sMAPE: 88.87%  |  MAE: 0.492
2022-01-23 - sMAP

2022-07-14 - sMAPE: 81.92%  |  MAE: 0.500
2022-07-15 - sMAPE: 82.07%  |  MAE: 0.500
2022-07-16 - sMAPE: 82.20%  |  MAE: 0.502
2022-07-17 - sMAPE: 82.25%  |  MAE: 0.507
2022-07-18 - sMAPE: 82.27%  |  MAE: 0.508
2022-07-19 - sMAPE: 82.59%  |  MAE: 0.511
2022-07-20 - sMAPE: 82.66%  |  MAE: 0.510
2022-07-21 - sMAPE: 82.77%  |  MAE: 0.509
2022-07-22 - sMAPE: 82.81%  |  MAE: 0.508
2022-07-23 - sMAPE: 83.31%  |  MAE: 0.511
2022-07-24 - sMAPE: 83.44%  |  MAE: 0.512
2022-07-25 - sMAPE: 83.53%  |  MAE: 0.513
2022-07-26 - sMAPE: 83.73%  |  MAE: 0.514
2022-07-27 - sMAPE: 83.67%  |  MAE: 0.513
2022-07-28 - sMAPE: 83.78%  |  MAE: 0.514
2022-07-29 - sMAPE: 84.23%  |  MAE: 0.516
2022-07-30 - sMAPE: 84.16%  |  MAE: 0.516
2022-07-31 - sMAPE: 84.05%  |  MAE: 0.518
2022-08-01 - sMAPE: 84.27%  |  MAE: 0.517
2022-08-02 - sMAPE: 84.32%  |  MAE: 0.517
2022-08-03 - sMAPE: 84.37%  |  MAE: 0.516
2022-08-04 - sMAPE: 84.35%  |  MAE: 0.515
2022-08-05 - sMAPE: 84.31%  |  MAE: 0.514
2022-08-06 - sMAPE: 84.25%  |  MAE

2023-01-26 - sMAPE: 84.83%  |  MAE: 0.521
2023-01-27 - sMAPE: 84.75%  |  MAE: 0.521
2023-01-28 - sMAPE: 84.68%  |  MAE: 0.520
2023-01-29 - sMAPE: 84.61%  |  MAE: 0.520
2023-01-30 - sMAPE: 84.50%  |  MAE: 0.520
2023-01-31 - sMAPE: 84.37%  |  MAE: 0.519
2023-02-01 - sMAPE: 84.39%  |  MAE: 0.519
2023-02-02 - sMAPE: 84.29%  |  MAE: 0.519
2023-02-03 - sMAPE: 84.17%  |  MAE: 0.518
2023-02-04 - sMAPE: 84.23%  |  MAE: 0.517
2023-02-05 - sMAPE: 84.18%  |  MAE: 0.518
2023-02-06 - sMAPE: 84.22%  |  MAE: 0.518
2023-02-07 - sMAPE: 84.20%  |  MAE: 0.517
2023-02-08 - sMAPE: 84.11%  |  MAE: 0.516
2023-02-09 - sMAPE: 84.06%  |  MAE: 0.516
2023-02-10 - sMAPE: 84.01%  |  MAE: 0.515
2023-02-11 - sMAPE: 84.01%  |  MAE: 0.514
2023-02-12 - sMAPE: 84.11%  |  MAE: 0.514
2023-02-13 - sMAPE: 84.07%  |  MAE: 0.513
2023-02-14 - sMAPE: 84.05%  |  MAE: 0.512
2023-02-15 - sMAPE: 84.05%  |  MAE: 0.512
2023-02-16 - sMAPE: 84.05%  |  MAE: 0.512
2023-02-17 - sMAPE: 84.01%  |  MAE: 0.512
2023-02-18 - sMAPE: 84.03%  |  MAE

In [13]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,-0.710291,-0.898288,-1.17568,-1.150891,-1.276044,-1.305522,-1.3715,-1.45493,-1.690259,-1.681645,...,-0.969086,-1.045576,-1.148349,-1.051055,-0.762817,-0.282995,-0.060811,0.101985,0.053608,-0.17996
2022-01-02,0.024746,-0.101846,-0.534454,-0.732823,-0.865671,-0.942755,-0.825589,-0.568109,-0.312203,-0.202156,...,0.000877,-0.096309,-0.210268,-0.025076,0.365788,0.731826,0.989668,1.007034,0.944428,0.690678
2022-01-03,0.527586,0.479771,0.187005,-0.005126,-0.093005,-0.092174,0.105959,0.610226,1.116092,1.295732,...,0.765835,0.607415,0.475725,0.433924,0.640198,0.87437,1.116788,1.116874,1.008793,0.662537
2022-01-04,-0.880447,-1.321633,-1.502582,-1.595879,-1.754272,-1.805969,-1.584153,-1.243382,-0.442973,0.005261,...,-0.745391,-0.939421,-0.9505,-0.858048,-0.500271,-0.059896,0.408708,0.398526,0.019368,-0.413914
2022-01-05,-0.783149,-1.107049,-1.42656,-1.539562,-1.636334,-1.638494,-1.443776,-0.889801,-0.041522,0.261275,...,-0.136025,-0.332229,-0.277552,-0.122632,0.325314,0.651024,0.929706,1.018922,0.784223,0.419323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-27,1.125817,1.060501,0.82535,0.73237,0.602659,0.562173,0.643642,0.748027,0.831358,0.757028,...,-0.025884,-0.191021,-0.466263,-0.515836,-0.424696,-0.057277,0.718783,1.218944,1.461908,1.348767
2023-05-28,1.223414,1.200948,0.999075,0.780118,0.666556,0.584491,0.692708,0.764017,0.690793,0.585639,...,-0.107993,-0.225752,-0.614741,-0.765756,-0.580974,-0.132851,0.530261,1.226764,1.519128,1.478369
2023-05-29,0.96501,0.940743,0.688883,0.458643,0.333422,0.367994,0.540615,0.932828,1.337565,1.387896,...,0.339975,0.223322,0.110572,0.12839,0.325393,0.585845,1.168293,1.647684,1.677394,1.463248


In [14]:
forecast.to_csv("Resultados/dnn_espana_estandarizacion_movil_final.csv")

In [15]:
df_test_original = pd.DataFrame(forecast.values.reshape((-1,)), columns=['Price'])
df_test_original['Date'] = df_scalers.tail(len(df_test_original)).Date.to_numpy()

In [16]:
predicciones = np.array([])
for cont in range(0, len(df_test_original)):
    
    print(df_test_original.iloc[cont].Date)
    
    # Acumulamos las predicciones
    scaler_obj = df_scalers[df_scalers.Date == df_test_original.iloc[cont].Date].scaler.values[0]

    predicciones = np.append(predicciones, scaler_obj.inverse_transform(np.array(df_test_original.iloc[cont].Price).reshape(1, -1))[0][0])

2022-01-01 00:00:00
2022-01-01 01:00:00
2022-01-01 02:00:00
2022-01-01 03:00:00
2022-01-01 04:00:00
2022-01-01 05:00:00
2022-01-01 06:00:00
2022-01-01 07:00:00
2022-01-01 08:00:00
2022-01-01 09:00:00
2022-01-01 10:00:00
2022-01-01 11:00:00
2022-01-01 12:00:00
2022-01-01 13:00:00
2022-01-01 14:00:00
2022-01-01 15:00:00
2022-01-01 16:00:00
2022-01-01 17:00:00
2022-01-01 18:00:00
2022-01-01 19:00:00
2022-01-01 20:00:00
2022-01-01 21:00:00
2022-01-01 22:00:00
2022-01-01 23:00:00
2022-01-02 00:00:00
2022-01-02 01:00:00
2022-01-02 02:00:00
2022-01-02 03:00:00
2022-01-02 04:00:00
2022-01-02 05:00:00
2022-01-02 06:00:00
2022-01-02 07:00:00
2022-01-02 08:00:00
2022-01-02 09:00:00
2022-01-02 10:00:00
2022-01-02 11:00:00
2022-01-02 12:00:00
2022-01-02 13:00:00
2022-01-02 14:00:00
2022-01-02 15:00:00
2022-01-02 16:00:00
2022-01-02 17:00:00
2022-01-02 18:00:00
2022-01-02 19:00:00
2022-01-02 20:00:00
2022-01-02 21:00:00
2022-01-02 22:00:00
2022-01-02 23:00:00
2022-01-03 00:00:00
2022-01-03 01:00:00


2022-01-20 12:00:00
2022-01-20 13:00:00
2022-01-20 14:00:00
2022-01-20 15:00:00
2022-01-20 16:00:00
2022-01-20 17:00:00
2022-01-20 18:00:00
2022-01-20 19:00:00
2022-01-20 20:00:00
2022-01-20 21:00:00
2022-01-20 22:00:00
2022-01-20 23:00:00
2022-01-21 00:00:00
2022-01-21 01:00:00
2022-01-21 02:00:00
2022-01-21 03:00:00
2022-01-21 04:00:00
2022-01-21 05:00:00
2022-01-21 06:00:00
2022-01-21 07:00:00
2022-01-21 08:00:00
2022-01-21 09:00:00
2022-01-21 10:00:00
2022-01-21 11:00:00
2022-01-21 12:00:00
2022-01-21 13:00:00
2022-01-21 14:00:00
2022-01-21 15:00:00
2022-01-21 16:00:00
2022-01-21 17:00:00
2022-01-21 18:00:00
2022-01-21 19:00:00
2022-01-21 20:00:00
2022-01-21 21:00:00
2022-01-21 22:00:00
2022-01-21 23:00:00
2022-01-22 00:00:00
2022-01-22 01:00:00
2022-01-22 02:00:00
2022-01-22 03:00:00
2022-01-22 04:00:00
2022-01-22 05:00:00
2022-01-22 06:00:00
2022-01-22 07:00:00
2022-01-22 08:00:00
2022-01-22 09:00:00
2022-01-22 10:00:00
2022-01-22 11:00:00
2022-01-22 12:00:00
2022-01-22 13:00:00


2022-02-07 13:00:00
2022-02-07 14:00:00
2022-02-07 15:00:00
2022-02-07 16:00:00
2022-02-07 17:00:00
2022-02-07 18:00:00
2022-02-07 19:00:00
2022-02-07 20:00:00
2022-02-07 21:00:00
2022-02-07 22:00:00
2022-02-07 23:00:00
2022-02-08 00:00:00
2022-02-08 01:00:00
2022-02-08 02:00:00
2022-02-08 03:00:00
2022-02-08 04:00:00
2022-02-08 05:00:00
2022-02-08 06:00:00
2022-02-08 07:00:00
2022-02-08 08:00:00
2022-02-08 09:00:00
2022-02-08 10:00:00
2022-02-08 11:00:00
2022-02-08 12:00:00
2022-02-08 13:00:00
2022-02-08 14:00:00
2022-02-08 15:00:00
2022-02-08 16:00:00
2022-02-08 17:00:00
2022-02-08 18:00:00
2022-02-08 19:00:00
2022-02-08 20:00:00
2022-02-08 21:00:00
2022-02-08 22:00:00
2022-02-08 23:00:00
2022-02-09 00:00:00
2022-02-09 01:00:00
2022-02-09 02:00:00
2022-02-09 03:00:00
2022-02-09 04:00:00
2022-02-09 05:00:00
2022-02-09 06:00:00
2022-02-09 07:00:00
2022-02-09 08:00:00
2022-02-09 09:00:00
2022-02-09 10:00:00
2022-02-09 11:00:00
2022-02-09 12:00:00
2022-02-09 13:00:00
2022-02-09 14:00:00


2022-02-25 18:00:00
2022-02-25 19:00:00
2022-02-25 20:00:00
2022-02-25 21:00:00
2022-02-25 22:00:00
2022-02-25 23:00:00
2022-02-26 00:00:00
2022-02-26 01:00:00
2022-02-26 02:00:00
2022-02-26 03:00:00
2022-02-26 04:00:00
2022-02-26 05:00:00
2022-02-26 06:00:00
2022-02-26 07:00:00
2022-02-26 08:00:00
2022-02-26 09:00:00
2022-02-26 10:00:00
2022-02-26 11:00:00
2022-02-26 12:00:00
2022-02-26 13:00:00
2022-02-26 14:00:00
2022-02-26 15:00:00
2022-02-26 16:00:00
2022-02-26 17:00:00
2022-02-26 18:00:00
2022-02-26 19:00:00
2022-02-26 20:00:00
2022-02-26 21:00:00
2022-02-26 22:00:00
2022-02-26 23:00:00
2022-02-27 00:00:00
2022-02-27 01:00:00
2022-02-27 02:00:00
2022-02-27 03:00:00
2022-02-27 04:00:00
2022-02-27 05:00:00
2022-02-27 06:00:00
2022-02-27 07:00:00
2022-02-27 08:00:00
2022-02-27 09:00:00
2022-02-27 10:00:00
2022-02-27 11:00:00
2022-02-27 12:00:00
2022-02-27 13:00:00
2022-02-27 14:00:00
2022-02-27 15:00:00
2022-02-27 16:00:00
2022-02-27 17:00:00
2022-02-27 18:00:00
2022-02-27 19:00:00


2022-03-16 09:00:00
2022-03-16 10:00:00
2022-03-16 11:00:00
2022-03-16 12:00:00
2022-03-16 13:00:00
2022-03-16 14:00:00
2022-03-16 15:00:00
2022-03-16 16:00:00
2022-03-16 17:00:00
2022-03-16 18:00:00
2022-03-16 19:00:00
2022-03-16 20:00:00
2022-03-16 21:00:00
2022-03-16 22:00:00
2022-03-16 23:00:00
2022-03-17 00:00:00
2022-03-17 01:00:00
2022-03-17 02:00:00
2022-03-17 03:00:00
2022-03-17 04:00:00
2022-03-17 05:00:00
2022-03-17 06:00:00
2022-03-17 07:00:00
2022-03-17 08:00:00
2022-03-17 09:00:00
2022-03-17 10:00:00
2022-03-17 11:00:00
2022-03-17 12:00:00
2022-03-17 13:00:00
2022-03-17 14:00:00
2022-03-17 15:00:00
2022-03-17 16:00:00
2022-03-17 17:00:00
2022-03-17 18:00:00
2022-03-17 19:00:00
2022-03-17 20:00:00
2022-03-17 21:00:00
2022-03-17 22:00:00
2022-03-17 23:00:00
2022-03-18 00:00:00
2022-03-18 01:00:00
2022-03-18 02:00:00
2022-03-18 03:00:00
2022-03-18 04:00:00
2022-03-18 05:00:00
2022-03-18 06:00:00
2022-03-18 07:00:00
2022-03-18 08:00:00
2022-03-18 09:00:00
2022-03-18 10:00:00


...

2023-03-06 19:00:00
2023-03-06 20:00:00
2023-03-06 21:00:00
2023-03-06 22:00:00
2023-03-06 23:00:00
2023-03-07 00:00:00
2023-03-07 01:00:00
2023-03-07 02:00:00
2023-03-07 03:00:00
2023-03-07 04:00:00
2023-03-07 05:00:00
2023-03-07 06:00:00
2023-03-07 07:00:00
2023-03-07 08:00:00
2023-03-07 09:00:00
2023-03-07 10:00:00
2023-03-07 11:00:00
2023-03-07 12:00:00
2023-03-07 13:00:00
2023-03-07 14:00:00
2023-03-07 15:00:00
2023-03-07 16:00:00
2023-03-07 17:00:00
2023-03-07 18:00:00
2023-03-07 19:00:00
2023-03-07 20:00:00
2023-03-07 21:00:00
2023-03-07 22:00:00
2023-03-07 23:00:00
2023-03-08 00:00:00
2023-03-08 01:00:00
2023-03-08 02:00:00
2023-03-08 03:00:00
2023-03-08 04:00:00
2023-03-08 05:00:00
2023-03-08 06:00:00
2023-03-08 07:00:00
2023-03-08 08:00:00
2023-03-08 09:00:00
2023-03-08 10:00:00
2023-03-08 11:00:00
2023-03-08 12:00:00
2023-03-08 13:00:00
2023-03-08 14:00:00
2023-03-08 15:00:00
2023-03-08 16:00:00
2023-03-08 17:00:00
2023-03-08 18:00:00
2023-03-08 19:00:00
2023-03-08 20:00:00


2023-03-24 17:00:00
2023-03-24 18:00:00
2023-03-24 19:00:00
2023-03-24 20:00:00
2023-03-24 21:00:00
2023-03-24 22:00:00
2023-03-24 23:00:00
2023-03-25 00:00:00
2023-03-25 01:00:00
2023-03-25 02:00:00
2023-03-25 03:00:00
2023-03-25 04:00:00
2023-03-25 05:00:00
2023-03-25 06:00:00
2023-03-25 07:00:00
2023-03-25 08:00:00
2023-03-25 09:00:00
2023-03-25 10:00:00
2023-03-25 11:00:00
2023-03-25 12:00:00
2023-03-25 13:00:00
2023-03-25 14:00:00
2023-03-25 15:00:00
2023-03-25 16:00:00
2023-03-25 17:00:00
2023-03-25 18:00:00
2023-03-25 19:00:00
2023-03-25 20:00:00
2023-03-25 21:00:00
2023-03-25 22:00:00
2023-03-25 23:00:00
2023-03-26 00:00:00
2023-03-26 01:00:00
2023-03-26 02:00:00
2023-03-26 03:00:00
2023-03-26 04:00:00
2023-03-26 05:00:00
2023-03-26 06:00:00
2023-03-26 07:00:00
2023-03-26 08:00:00
2023-03-26 09:00:00
2023-03-26 10:00:00
2023-03-26 11:00:00
2023-03-26 12:00:00
2023-03-26 13:00:00
2023-03-26 14:00:00
2023-03-26 15:00:00
2023-03-26 16:00:00
2023-03-26 17:00:00
2023-03-26 18:00:00


2023-04-11 11:00:00
2023-04-11 12:00:00
2023-04-11 13:00:00
2023-04-11 14:00:00
2023-04-11 15:00:00
2023-04-11 16:00:00
2023-04-11 17:00:00
2023-04-11 18:00:00
2023-04-11 19:00:00
2023-04-11 20:00:00
2023-04-11 21:00:00
2023-04-11 22:00:00
2023-04-11 23:00:00
2023-04-12 00:00:00
2023-04-12 01:00:00
2023-04-12 02:00:00
2023-04-12 03:00:00
2023-04-12 04:00:00
2023-04-12 05:00:00
2023-04-12 06:00:00
2023-04-12 07:00:00
2023-04-12 08:00:00
2023-04-12 09:00:00
2023-04-12 10:00:00
2023-04-12 11:00:00
2023-04-12 12:00:00
2023-04-12 13:00:00
2023-04-12 14:00:00
2023-04-12 15:00:00
2023-04-12 16:00:00
2023-04-12 17:00:00
2023-04-12 18:00:00
2023-04-12 19:00:00
2023-04-12 20:00:00
2023-04-12 21:00:00
2023-04-12 22:00:00
2023-04-12 23:00:00
2023-04-13 00:00:00
2023-04-13 01:00:00
2023-04-13 02:00:00
2023-04-13 03:00:00
2023-04-13 04:00:00
2023-04-13 05:00:00
2023-04-13 06:00:00
2023-04-13 07:00:00
2023-04-13 08:00:00
2023-04-13 09:00:00
2023-04-13 10:00:00
2023-04-13 11:00:00
2023-04-13 12:00:00


2023-04-29 22:00:00
2023-04-29 23:00:00
2023-04-30 00:00:00
2023-04-30 01:00:00
2023-04-30 02:00:00
2023-04-30 03:00:00
2023-04-30 04:00:00
2023-04-30 05:00:00
2023-04-30 06:00:00
2023-04-30 07:00:00
2023-04-30 08:00:00
2023-04-30 09:00:00
2023-04-30 10:00:00
2023-04-30 11:00:00
2023-04-30 12:00:00
2023-04-30 13:00:00
2023-04-30 14:00:00
2023-04-30 15:00:00
2023-04-30 16:00:00
2023-04-30 17:00:00
2023-04-30 18:00:00
2023-04-30 19:00:00
2023-04-30 20:00:00
2023-04-30 21:00:00
2023-04-30 22:00:00
2023-04-30 23:00:00
2023-05-01 00:00:00
2023-05-01 01:00:00
2023-05-01 02:00:00
2023-05-01 03:00:00
2023-05-01 04:00:00
2023-05-01 05:00:00
2023-05-01 06:00:00
2023-05-01 07:00:00
2023-05-01 08:00:00
2023-05-01 09:00:00
2023-05-01 10:00:00
2023-05-01 11:00:00
2023-05-01 12:00:00
2023-05-01 13:00:00
2023-05-01 14:00:00
2023-05-01 15:00:00
2023-05-01 16:00:00
2023-05-01 17:00:00
2023-05-01 18:00:00
2023-05-01 19:00:00
2023-05-01 20:00:00
2023-05-01 21:00:00
2023-05-01 22:00:00
2023-05-01 23:00:00


2023-05-18 12:00:00
2023-05-18 13:00:00
2023-05-18 14:00:00
2023-05-18 15:00:00
2023-05-18 16:00:00
2023-05-18 17:00:00
2023-05-18 18:00:00
2023-05-18 19:00:00
2023-05-18 20:00:00
2023-05-18 21:00:00
2023-05-18 22:00:00
2023-05-18 23:00:00
2023-05-19 00:00:00
2023-05-19 01:00:00
2023-05-19 02:00:00
2023-05-19 03:00:00
2023-05-19 04:00:00
2023-05-19 05:00:00
2023-05-19 06:00:00
2023-05-19 07:00:00
2023-05-19 08:00:00
2023-05-19 09:00:00
2023-05-19 10:00:00
2023-05-19 11:00:00
2023-05-19 12:00:00
2023-05-19 13:00:00
2023-05-19 14:00:00
2023-05-19 15:00:00
2023-05-19 16:00:00
2023-05-19 17:00:00
2023-05-19 18:00:00
2023-05-19 19:00:00
2023-05-19 20:00:00
2023-05-19 21:00:00
2023-05-19 22:00:00
2023-05-19 23:00:00
2023-05-20 00:00:00
2023-05-20 01:00:00
2023-05-20 02:00:00
2023-05-20 03:00:00
2023-05-20 04:00:00
2023-05-20 05:00:00
2023-05-20 06:00:00
2023-05-20 07:00:00
2023-05-20 08:00:00
2023-05-20 09:00:00
2023-05-20 10:00:00
2023-05-20 11:00:00
2023-05-20 12:00:00
2023-05-20 13:00:00


In [17]:
df_test_original['Price'] = predicciones

In [18]:
df_test_original = df_test_original.set_index(df_test_original.Date).drop('Date', axis = 1)

In [19]:
forecast_final = df_test_original.loc[:, ['Price']].values.reshape(-1, 24)
forecast_final = pd.DataFrame(forecast_final, index=df_test_original.index[::24], columns=['h' + str(k) for k in range(24)])

In [20]:
forecast_final

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,117.303882,105.107915,87.112543,88.720725,80.601588,78.689247,74.409053,68.996705,53.730116,54.288931,...,100.514982,95.552829,88.885596,95.197405,113.896379,145.024008,159.437870,169.998933,166.860562,151.708246
2022-01-02,149.772973,142.290745,116.721466,104.996906,97.144922,92.588884,99.513951,114.732336,129.857633,136.361930,...,148.362206,142.618037,135.882501,146.828263,169.930233,191.564898,206.804611,207.831044,204.130712,189.132812
2022-01-03,170.189915,167.482199,150.902888,140.022491,135.045923,135.093010,146.313228,174.869813,203.516967,213.689957,...,183.681965,174.710618,167.253070,164.885879,176.567128,189.828293,203.556358,203.561266,197.440644,177.832220
2022-01-04,105.344686,83.882171,75.079529,70.540856,62.835511,60.320599,71.111301,87.688890,126.626600,148.431935,...,111.914799,102.475758,101.936789,106.434327,123.839217,145.262243,168.058523,167.563197,149.118208,128.040227
2022-01-05,121.890420,108.194117,94.683453,89.905094,85.813039,85.721689,93.955436,117.380581,153.250510,166.054483,...,149.254406,140.957838,143.269874,149.820767,168.762376,182.535207,194.319448,198.091972,188.167624,172.737623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-27,102.189782,100.201898,93.045105,90.215266,86.267542,85.035372,87.514872,90.691808,93.227970,90.965732,...,67.137928,62.112018,53.735069,52.226325,55.000158,66.182483,89.801758,105.024077,112.418667,108.975234
2023-05-28,107.829629,107.201972,101.561878,95.444473,92.271699,89.978895,93.002363,94.994648,92.948856,90.010970,...,70.631735,67.341698,56.473814,52.254633,57.417229,69.937246,88.463765,107.923225,116.091517,114.952764
2023-05-29,101.027316,100.533146,95.404276,90.715652,88.165652,88.869679,92.384939,100.371979,108.614039,109.638996,...,88.299095,85.923569,83.627530,83.990364,88.002146,93.306007,105.166976,114.929326,115.534336,111.173460


In [21]:
forecast_final.to_csv("Resultados/dnn_espana_estandarizacion_movil_final.csv")

In [28]:
_, df_test = read_data(dataset="df_espana_dnn", years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00


In [29]:
mae = np.mean(MAE(forecast_final.values.squeeze(), real_values.values)) 
smape = np.mean(sMAPE(forecast_final.values.squeeze(), real_values.values)) * 100
print('sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(smape, mae))

sMAPE: 20.67%  |  MAE: 17.928
